In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import sys
import os
import time


import mlflow
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import xgboost as xgb
from lightgbm import LGBMClassifier

In [2]:
df1 = pd.read_excel("train4_outliers_treated.xlsx")
df1.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,PrizmCode_Rural,PrizmCode_Suburban,PrizmCode_Town,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student
0,-1.034271,-0.641018,-1.132206,-0.405699,-0.697466,-0.597359,-0.993339,-1.667634,-0.800615,-0.571428,...,-0.224589,1.449140,-0.421117,-0.176738,-0.056264,-1.655607,2.175118,-0.12169,-0.133565,-0.087381
1,-1.266366,-1.122170,-1.365359,-0.704544,-0.697466,-0.597359,0.031072,0.199780,-0.873059,-0.798276,...,-0.224589,1.449140,-0.421117,-0.176738,-0.056264,-1.655607,2.175118,-0.12169,-0.133565,-0.087381
2,-0.570745,-1.126774,-0.386116,-0.704544,-0.697466,-0.597359,0.044463,0.199780,-0.927391,-0.798276,...,-0.224589,-0.690065,2.374639,5.658086,-0.056264,-1.655607,-0.459745,-0.12169,-0.133565,-0.087381
3,0.895322,1.875243,1.339217,0.777726,-0.697466,-0.597359,1.109047,0.995888,2.368792,1.697050,...,-0.224589,-0.690065,-0.421117,-0.176738,-0.056264,0.604008,-0.459745,-0.12169,-0.133565,-0.087381
4,-1.261399,-1.145191,-1.365359,-0.704544,-0.697466,-0.597359,0.057854,0.180123,-0.927391,-0.798276,...,-0.224589,-0.690065,-0.421117,-0.176738,-0.056264,-1.655607,2.175118,-0.12169,-0.133565,-0.087381


In [4]:
# dropping the customerid
# df1.drop(['CustomerID'], axis=1, inplace=True)
# df1.head()

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve


# split the dataset into independent and dependent data
X = df1.drop('Churn', axis=1)
Y = df1['Churn']


# performing the split
x_train, x_test, y_train, y_test = train_test_split(X, Y, 
                                                   test_size=0.30,
                                                   random_state=1,
                                                   stratify = Y)

# Functions for Model Performance

In [ ]:
def model_performance(model_name,
                      x_train_data,
                      y_train_data, 
                      x_test_data,
                      y_test_data,): 

    train_preds = model_name.predict(x_train)
    test_preds= model_name.predict(x_test)
    
    a = classification_report(y_train_data, train_preds)
    b = classification_report(y_test_data, test_preds)
    
    c = round(model_name.score(x_train_data, y_train_data),4)
    d = round(model_name.score(x_test_data, y_test_data),4)
    
    print('classification report for training data')
    print(a)
    print('\n')
    print('classification report for testing data')
    print(b)
    print('\n')
    print('the model score for training data is ',c)
    print('the model score for testing data is ',d)
    print('\n')
    
    
    from seaborn import heatmap
    fig, axes = plt.subplots(ncols=2)
    fig.set_size_inches(10,5)

    sns.heatmap(confusion_matrix(y_test_data, test_preds), annot=True, cbar=False, fmt='d')
    plt.xlabel('predicted labels')
    plt.ylabel('actual labels')
    plt.title('Confusion Matrix for Testing Data')
    plt.subplot(1,2,1)

    sns.heatmap(confusion_matrix(y_train_data, train_preds), annot=True, cbar=False, fmt='d')
    plt.xlabel('predicted labels')
    plt.ylabel('actual labels')
    plt.title('Confusion Matrix for Training Data')
    plt.subplot(1,2,2)

In [ ]:
# ROC AUC Plots
def roc_score_auc_curve(model_name,
                       x_train_data,
                       y_train_data, 
                       x_test_data,
                       y_test_data):
    
    train_preds = model_name.predict(x_train)
    test_preds= model_name.predict(x_test)

    
    a = round(roc_auc_score(y_train_data,model_name.predict_proba(x_train_data)[:,1]),4)
    b = round(roc_auc_score(y_test,model_name.predict_proba(x_test_data)[:,1]),4)
    print('AUC Score for Model on Training Data is',a)
    print('AUC Score for Model on Testing Data is',b)
    
    plt.figure(figsize=(12,7))
    
    train_fpr, train_tpr, train_thresholds = roc_curve(y_train_data, model_name.predict_proba(x_train_data)[:,1])
    plt.plot([0,1],[0,1], linestyle='--', color='g')
    plt.plot(train_fpr, train_tpr, marker='.') 
  

    test_fpr, test_tpr, test_thresholds = roc_curve(y_test_data, model_name.predict_proba(x_test_data)[:,1])
    plt.plot([0,1],[0,1], linestyle='--', color='g')
    plt.plot(test_fpr, test_tpr, marker='o') 
    
# roc_score_auc_curve(mod7, x_train, y_train, train_pred_mod7, x_test, y_test, test_pred_mod7)

In [ ]:
# K-Fold Cross Validation Functions

def k_fold_cross_valscore(model_name,
                          x_train_data,
                          y_train_data,
                          folds
                         ):
    from sklearn.model_selection import cross_val_score
    import pandas as pd
    
    model_kfold_recall = cross_val_score(model_name, x_train_data, y_train_data,cv=folds, scoring='recall', verbose=0)
    model_kfold_accuracy = cross_val_score(model_name, x_train_data, y_train_data,cv=folds, scoring='accuracy', verbose=0)
    model_kfold_precision = cross_val_score(model_name, x_train_data, y_train_data,cv=folds, scoring='precision', verbose=0)
    model_kfold_f1 = cross_val_score(model_name, x_train_data, y_train_data,cv=folds, scoring='f1', verbose=0)
    
    a = pd.DataFrame(model_kfold_recall)
    b = pd.DataFrame(model_kfold_accuracy)
    c = pd.DataFrame(model_kfold_precision)
    d = pd.DataFrame(model_kfold_f1)

    cross_val_data = pd.concat([a,b,c,d], ignore_index=True, axis=1)
    cross_val_data.rename(columns={0:"Recall", 1:"Accuracy", 2:"Precision",3:"F1 Score"},
                        inplace=True)
       
   
    print('\n')
    print("The mean recall for the model after " ,folds," folds is ", np.mean(model_kfold_recall))
    print("The mean accuracy for model after ",folds," folds is", np.mean(model_kfold_accuracy))
    print("the mean precision for the model after ",folds," folds is",np.mean(model_kfold_precision))
    print("the mean f1 score for the model after ",folds," folds is", np.mean(model_kfold_f1))
    print("\n")
    return cross_val_data

# k_fold_cross_valscore(mod7, x_train, y_train, folds=10)

## Base Models

In [ ]:
# Setting up a dictionary
base_models = {
    'base LDA Model' : LinearDiscriminantAnalysis(),
    'base Logistic Regression Model' : LogisticRegression(random_state=1, max_iter=100000),
    'base ADA Boost Model' : AdaBoostClassifier(random_state=1),
    'base Decision Tree Model' : DecisionTreeClassifier(random_state=1, criterion='gini'),
    'base Random Forest Model' : RandomForestClassifier(random_state=1),
    'base KNN Model' : KNeighborsClassifier(),
    'base XGB Classifier' : xgb.XGBClassifier(random_state=1, use_label_encoder=False),
    'base MLP Classifier' : MLPClassifier(random_state=1, max_iter = 5000),
    'base QDA Model' : QuadraticDiscriminantAnalysis()
}

In [ ]:
# function for training a model
def model_training(model, x_train, y_train):
    # Train the Model on training data
    trained_model = model.fit(x_train, y_train)
    return trained_model



for model_name,model in base_models.items():
    print(model_name.upper())
    
    # Model Training
    trained_model = model_training(model, x_train, y_train)
    
    # # train and test preds
    train_pred_mod = trained_model.predict(x_train)
    test_pred_mod = trained_model.predict(x_test)

    
    # # look at the model performance metrics
    model_performance(trained_model , x_train, y_train, train_pred_mod, x_test, y_test, test_pred_mod)


    # # ROC-AUC 
    roc_score_auc_curve(trained_model, x_train, y_train, train_pred_mod, x_test, y_test, test_pred_mod)

    # Display the plots
    plt.show()

    
    print('\n')
    print('######################################################################################################################################################################################################')
    print('\n')

## 1. Tuning LDA Model

In [ ]:
params = {
    'solver' : ['svd', 'lsqr', 'eigen'],
    # 'tol' : [0.001,0.0001,0.00001]
}

lda_model =  LinearDiscriminantAnalysis()
grid = GridSearchCV(estimator = lda_model, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 2. Logistic Regression

In [ ]:
params = {
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'tol' : [0.01,0.001,0.0001],
    #'penalty': ['l1', 'l2'],
    # 'max_iter': [1000, 10000],
    'C' : np.logspace(-4,4,10,20),
}
logit_model =  LogisticRegression(max_iter = 100000,random_state=1, class_weight='balanced')
grid = GridSearchCV(estimator = logit_model, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 3. ADA Boost Model

In [ ]:
params = {
    'n_estimators' : [500,600,700,800,900,1000,1100,1200,1300],
    'algorithm' : ['SAMME', 'SAMME.R'],
    'learning_rate' : [0.01,0.05,0.1,0.2,0.3,0.5]
   }
ada =  AdaBoostClassifier(random_state = 1)
grid = GridSearchCV(estimator = ada, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 4. Decision Tree Model

In [ ]:
params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [6,7,8,9,10,11,12,13,14],
    #'min_samples_leaf' : [15,20,25,30,35,40,45,50,55,60,65,70],
    #'min_samples_split' : [25,30,35,40,45,50,60,70,80,90,100],
    }

dt_model = DecisionTreeClassifier(random_state=1)

grid_search = GridSearchCV(estimator=dt_model, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 5. Random Forest Model

In [ ]:
params = {
    'max_depth' : [7,8,9,10,11,12,13,14],
    'max_features': [15,16,17,18],
    #'min_samples_leaf': [15,20,25,30],
    #'min_samples_split': [60,65,70],
    'n_estimators' : [401,501,601,701,801],
}

rfcl = RandomForestClassifier()
grid = GridSearchCV(estimator=rfcl, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 6. KNN Model

In [ ]:
params = {
    'n_neighbors' : [2,3,5,7,],  #9,11
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size' : [15,20,25,30,35],
    'metric' :['minkowski','euclidean','manhattan','chebyshev']
    }
knn_model =  KNeighborsClassifier(n_jobs=-1)
grid = GridSearchCV(estimator = knn_model, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 7. XGBoost Model

In [ ]:
params = {
    'colsample_bytree': [0.5, 0.7], 
    'learning_rate': [0.1,0.5],      
    'max_depth': [7,8,9,10],         
    'n_estimators' : [800,900,1000,1100 ,1200],
    'subsample': [0.3,0.5, 0.7],
    
    }
   
xgb =  xgb.XGBClassifier(random_state = 1, use_label_encoder=False, booster='gbtree', eval_metric='logloss')
grid = GridSearchCV(estimator = xgb, param_grid=params, cv=3, scoring='recall', verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(best_model, x_train, y_train, x_test, y_test)

## 8. ANN Model

In [ ]:
params = {
    'activation': ['logistic', 'relu', 'tanh', 'identity' ],
    'hidden_layer_sizes': [100, 200, 300, (100,100), (200,200), (300,300), (100, 100, 100)],
    'max_iter' : [10000],
    # 'solver': ['sgd', 'adam', 'lbfgs'],
    #'tol': [0.1,0.5,0.01],
    
}

ann = MLPClassifier(random_state=1)
grid = GridSearchCV(estimator=ann, param_grid=params,scoring='recall', cv=3, verbose=3)

In [ ]:
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
model_performance(best_model , x_train, y_train, x_test, y_test)

## 9. Quadratic Discriminant Analysis

In [ ]:
model = QuadraticDiscriminantAnalysis()
qda_model = model.fit(x_train, y_train)
qda_model

In [ ]:
model_performance(qda_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(qda_model, x_train, y_train, x_test, y_test)

## Light Gradient Boosting

In [ ]:
# Initialize a LightGBM Classifier with 'auc' as the evaluation metric 
lgb_model = LGBMClassifier(metric='recall') 
  
# Fit the model on the training data 
lgb_model.fit(x_train, y_train) 
  
# # Make predictions on the training and validation sets 
# y_train = model.predict(X_train) 
# y_val = model.predict(X_val) 

In [ ]:
model_performance(lgb_model , x_train, y_train, x_test, y_test)

In [ ]:
roc_score_auc_curve(lgb_model, x_train, y_train, x_test, y_test)